In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime
from typing import Dict, Tuple, Optional

# --- CONFIGURATION ---

# ABSOLUTE PATH to the directory containing the raw CSV files from Chirascan.
# NOTE: We use the 'r' prefix (raw string) to handle backslashes in Windows paths correctly.
# Replace the path below with your actual local path.
FOLDER_PATH = r'C:\Users\Filip\Documents\Research\Biophysics_Project_G4\Raw_Spectra_Data'

# --- HELPER FUNCTIONS ---

def parse_chirascan_file(file_path: str) -> Tuple[Optional[str], Optional[pd.DataFrame]]:
    """
    Parses a single Chirascan CSV file to extract metadata (Date) and spectral data.
   
    Args:
        file_path (str): The absolute path to the CSV file.
       
    Returns:
        Tuple[str, pd.DataFrame]: A tuple containing the measurement date and the data.
                                  Returns (None, None) if parsing fails.
    """
    try:
        # Open the file safely to read metadata before loading the bulk data with Pandas.
        # 'errors="ignore"' is used to bypass potential encoding issues in older instrument files.
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()

        # 1. METADATA EXTRACTION
        # Locate the line containing the date (Format in file: "#Date: YYYY/MM/DD")
        # We use a generator expression for memory efficiency.
        date_line = next((line for line in lines if '#Date:' in line), None)
       
        # If date is missing, mark as 'Unknown', otherwise strip whitespace.
        measurement_date = date_line.split('#Date:')[-1].strip() if date_line else 'Unknown_Date'

        # 2. DYNAMIC HEADER DETECTION
        # The data section starts after the 'Data:' keyword.
        # We need to find the exact line where numerical values begin.
        start_line_idx = None
        for i, line in enumerate(lines):
            if 'Data:' in line:
                # Look ahead to find the first line containing a decimal point (indicating numbers).
                # This makes the parser robust against varying header lengths.
                start_line_idx = next((j for j, l in enumerate(lines[i+1:], start=i+1) if '.' in l), None)
                break
       
        # Safety check: If no data start is found, log a warning and skip.
        if start_line_idx is None:
            print(f"Warning: Could not identify the data start line in file: {os.path.basename(file_path)}")
            return None, None

        # Find the end of the data block (first empty line after data starts).
        end_line_idx = next((i for i, line in enumerate(lines[start_line_idx:], start=start_line_idx) if not line.strip()), None)
       
        # Calculate the number of rows to read.
        rows_to_read = (end_line_idx - start_line_idx) if end_line_idx else None

        # 3. DATA LOADING
        # Load only the relevant section into a DataFrame.
        # We explicitly select columns 0 (Wavelength) and 1 (Signal/Ellipticity).
        df = pd.read_csv(
            file_path,
            header=None,
            skiprows=start_line_idx,
            nrows=rows_to_read,
            usecols=[0, 1]
        )
       
        # Handle empty files
        if df.empty:
            return None, None
           
        # Standardize column names for downstream processing
        df.columns = ['Wavelength', 'Signal']
       
        return measurement_date, df

    except Exception as e:
        # Catch-all for IO errors or unexpected formats to prevent the pipeline from crashing.
        print(f"Error processing file {os.path.basename(file_path)}: {e}")
        return None, None

def load_and_group_spectra(folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    Iterates through all CSV files in the target folder, parses them,
    and aggregates them by measurement date.
   
    Returns:
        Dict: Keys are dates (str), Values are DataFrames with merged spectra.
    """
    # Use glob to find all CSV files in the directory
    search_pattern = os.path.join(folder_path, "*.csv")
    files = glob.glob(search_pattern)
   
    if not files:
        print(f"No CSV files found in directory: {folder_path}")
        return {}

    # Temporary storage: { '2023/10/20': [ (filename1, df1), (filename2, df2) ] }
    data_buffer = {}

    print(f"Initializing batch processing for {len(files)} files...")

    # --- STEP 1: PARSING LOOP ---
    for file_path in files:
        file_name = os.path.basename(file_path).replace('.csv', '')
       
        # Parse individual file
        date, df = parse_chirascan_file(file_path)

        if df is not None:
            if date not in data_buffer:
                data_buffer[date] = []
            data_buffer[date].append((file_name, df))

    # --- STEP 2: AGGREGATION ---
    grouped_data = {}
   
    for date, file_list in data_buffer.items():
        # We use the Wavelength from the first file as the index (reference X-axis).
        # It is assumed all files share the same wavelength range.
        base_df = file_list[0][1][['Wavelength']].copy()
        base_df.set_index('Wavelength', inplace=True)
       
        all_signals = []
        for name, df in file_list:
            # Rename the 'Signal' column to the file name to keep track of samples.
            signal_series = df.set_index('Wavelength')['Signal']
            signal_series.name = name
            all_signals.append(signal_series)
           
        # Concatenate all series horizontally (axis=1) - Highly efficient operation in Pandas.
        merged_df = pd.concat(all_signals, axis=1)
       
        # Reset index to make Wavelength a regular column again (better for plotting/export).
        merged_df.reset_index(inplace=True)
        grouped_data[date] = merged_df
       
    print(f"Successfully grouped data into {len(grouped_data)} unique dates.")
    return grouped_data

def export_to_excel(grouped_data: Dict[str, pd.DataFrame], output_path: str):
    """
    Exports the aggregated data to a multi-section Excel file.
    Data blocks are placed side-by-side for easy comparison in Excel.
    """
    if not grouped_data:
        print("No data to export.")
        return

    # Sort dates chronologically to ensure logical order in the report
    try:
        sorted_dates = sorted(grouped_data.keys(), key=lambda d: datetime.strptime(d, '%Y/%m/%d'))
    except ValueError:
        # Fallback for non-standard date formats
        sorted_dates = sorted(grouped_data.keys())

    print(f"Exporting data to: {output_path}")

    # Use 'openpyxl' engine to write to .xlsx
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        start_col = 0
        sheet_name = 'Spectra Analysis'
       
        # Initialize the sheet
        pd.DataFrame().to_excel(writer, sheet_name=sheet_name)
        sheet = writer.sheets[sheet_name]

        for date in sorted_dates:
            df = grouped_data[date]
           
            # Write the Header (Date) above the data block
            sheet.cell(row=1, column=start_col + 1).value = f"Measurement Date: {date}"
           
            # Write the DataFrame
            df.to_excel(writer, sheet_name=sheet_name, startrow=1, startcol=start_col, index=False)
           
            # Shift the starting column for the next block (Width of DF + 1 spacer column)
            start_col += len(df.columns) + 1
           
    print("✅ Export completed successfully.")

# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":
    # Check if the configured path exists
    if not os.path.exists(FOLDER_PATH):
        print(f"Error: The path '{FOLDER_PATH}' does not exist.")
        print("Please update the 'FOLDER_PATH' variable in the configuration section.")
    else:
        # 1. Process Data
        results = load_and_group_spectra(FOLDER_PATH)
       
        # 2. Export Results (Saved in the same folder as input data)
        output_file = os.path.join(FOLDER_PATH, 'Processed_Spectra_Report.xlsx')
        export_to_excel(results, output_file)